In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy.sparse as sp
from scipy.sparse.linalg import svds


In [20]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('/Users/zhaoxiangyu/Desktop/ml-100k/u.data', sep='\t', names=header)

In [21]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)

Number of users = 943 | Number of movies = 1682


In [22]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.20)

In [23]:
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [24]:
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

In [25]:
def non_zero_mean(x):
    return x[x != 0].mean()
def predict(ratings, similarity, type):
    if type == 'user':
        mean_user_rating = np.apply_along_axis(non_zero_mean, axis = 1, arr = ratings)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        ratings_diff = np.where(ratings_diff==-mean_user_rating[:, np.newaxis],0,ratings_diff)
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [26]:
pred=predict(train_data_matrix, user_similarity, type='user')

In [27]:
pred

array([[ 3.67053431,  3.54626467,  3.54573797, ...,  3.57195606,
         3.5733945 ,  3.57313471],
       [ 3.86732611,  3.71998066,  3.72370568, ...,  3.75341929,
         3.75438596,  3.75394061],
       [ 3.01880955,  2.87011565,  2.87029601, ...,  2.90383988,
         2.9047619 ,  2.90431627],
       ..., 
       [ 4.04574625,  3.90406357,  3.90414498, ...,  3.93625784,
         3.9375    ,  3.93706033],
       [ 4.3450027 ,  4.2073763 ,  4.20663321, ...,  4.23763089,
         4.23880597,  4.23839857],
       [ 3.53976858,  3.41495011,  3.41366808, ...,  3.44140999,
         3.44285714,  3.44255128]])

In [28]:
test=np.where(train_data_matrix != 0, 0, pred)
test

array([[ 0.        ,  0.        ,  0.        , ...,  3.57195606,
         3.5733945 ,  3.57313471],
       [ 0.        ,  3.71998066,  3.72370568, ...,  3.75341929,
         3.75438596,  3.75394061],
       [ 3.01880955,  2.87011565,  2.87029601, ...,  2.90383988,
         2.9047619 ,  2.90431627],
       ..., 
       [ 4.04574625,  3.90406357,  3.90414498, ...,  3.93625784,
         3.9375    ,  3.93706033],
       [ 4.3450027 ,  4.2073763 ,  4.20663321, ...,  4.23763089,
         4.23880597,  4.23839857],
       [ 3.53976858,  3.41495011,  3.41366808, ...,  3.44140999,
         3.44285714,  3.44255128]])

In [29]:
df2=pd.DataFrame(test)

In [30]:
def add_pred(x):
    return df2.iloc[int(x[0]-1),int(x[1]-1)]
pred_col = df.apply(add_pred, axis = 1)

In [31]:
#for index in df.index.values:
#    df.iloc[index, 4]=df2.iloc[df.iloc[index, 0]-1,df.iloc[index, 1]-1]

In [32]:
df["pred"] = pred_col

In [33]:
df3=df[df['pred']!=0]

In [34]:
REC=df3.sort_values('pred',ascending=False).groupby('user_id').head(5)

In [35]:
(REC.sort_values('user_id')['rating']>=3).sum()

4079

In [36]:
(REC.sort_values('user_id')['rating']>=3).sum()*1.0/len(REC.index)

0.90123729562527621

In [37]:
(REC.sort_values('user_id')['rating']>=4).sum()*1.0/len(REC.index)

0.70547945205479456

In [38]:
REC.sort_values('user_id',ascending=True)

,user_id,item_id,rating,timestamp,pred
59972,1,168,5,874965478,3.671143
34026,1,173,5,878541803,3.689796
6541,1,258,5,878873389,3.691530
5909,1,185,4,875072631,3.651584
3358,1,98,4,875072404,3.759429
35337,2,294,1,888551648,3.621709
55790,2,100,5,888552084,3.981505
39562,2,10,2,888551853,3.770535
43627,2,255,4,888551341,3.730987
73282,2,304,4,888979197,3.757458


In [39]:
##df_by_user_id=df.groupby('user_id').size()

In [40]:
##hot=df_by_user_id.sort_values(ascending=False)

In [41]:
##np.where(df_by_user_id<50)[0]

In [42]:
k=df.iloc[np.where(df['pred']>=4)[0],:]

In [43]:
len(np.where(k['rating']>=4)[0])*1.0/k.shape[0]

0.8049036777583187